In [42]:
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import datetime

In [43]:
df = pd.read_csv("../datasets/df_tratado_final.csv", sep = ",")

In [44]:
df.head()

,SAP+NOME,UF,Semana,Mes,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,...,CMV/QTD,Região,DPPsic,Cont,Produto,Correlacao,NObs,Grupo,Semana_Ano,PDia_Semana
0,T VISITA P BORDA DESIRE AZUL 6238,AL,1,1,2017,4.74,0.482330,48.233032,4.74000,7.0,...,1.777143,Nordeste,1,1,T VISITA P BORDA DESIRE AZUL 6238,-0.140144,26.0,7.0,2017-1,2017-01-02
1,T VISITA P BORDA DESIRE AGUA 6243,AL,1,1,2017,4.99,0.535633,53.563281,4.99000,7.0,...,1.710000,Nordeste,1,1,T VISITA P BORDA DESIRE AGUA 6243,0.000750,28.0,7.0,2017-1,2017-01-02
2,AMACIANTE COMFORT CONCENTRADO 500ML,AL,1,1,2017,7.99,0.404144,40.414445,7.96619,99.0,...,4.316162,Nordeste,0,1,AMACIANTE COMFORT CONCENTRADO 500ML,-0.537418,44.0,0.0,2017-1,2017-01-02
3,T VISITA P BORDA DESIRE AZUL 6238,AM,1,1,2017,4.74,0.587795,58.779525,4.74000,4.0,...,1.590000,Norte,1,1,T VISITA P BORDA DESIRE AZUL 6238,-0.672686,38.0,7.0,2017-1,2017-01-02
4,T VISITA P BORDA DESIRE AGUA 6243,AM,1,1,2017,4.99,0.621958,62.195832,4.99000,15.0,...,1.553333,Norte,1,1,T VISITA P BORDA DESIRE AGUA 6243,-0.184839,46.0,7.0,2017-1,2017-01-02


# SORTED()

In [45]:
lista_unicos = sorted(list(df["SAP+NOME"].unique()))
df["Grupo"] = np.nan
grupo = 0
thresh = 52

# While

In [46]:
while len(lista_unicos) > 0:
    item = lista_unicos[0] 
    similaridades = process.extract(item, lista_unicos)
    for i in range(len(similaridades)):
        if (similaridades[i][1] >= thresh):
            df.loc[df["SAP+NOME"] == similaridades[i][0], "Grupo"] = grupo
            lista_unicos.remove(similaridades[i][0])
    grupo += 1
        

## Validação

In [47]:
for i in range(len(df["Grupo"].unique())):
    print(f'grupo => {i}')
    print(df.loc[df["Grupo"] == i]["SAP+NOME"].unique())

grupo => 0
['AMACIANTE COMFORT CONCENTRADO 500ML'
 'AMACIANTE DOWNY LIRIOS DO CAMPO 500ML'
 'AMACIANTE COMFORT PURO CUIDADO 500ML']
grupo => 1
['KIT CABIDE C/3 MADEIRA REV' 'CIF ULT RAP TIRA LIMO C/CLORO SQZ 500ML'
 'CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML']
grupo => 2
['DESINF LIQ PINHO SOL CITRUS LIMAO 500ML'
 'DESINF LIQ PINHO SOL ORIGINAL 500ML']
grupo => 3
['FRONHA AVULSA ESTAMPADA ANDREZA' 'FRONHA AVULSA LISA ANDREZA'
 'FRONHA AVULSA LISA CAMESA' 'FRONHA AVULSA ESTAMPADA CAMESA']
grupo => 4
['JG CAMA QUEEN PRATA 150FIOS MALIBU SANTI'
 'JG CAMA QUEEN MICROFIBRA ANDREZA']
grupo => 5
['PANO SCOTT DURAMAX LIMPEZA DIARIA C/5']
grupo => 6
['T VISITA P BORDA DESIRE AGUA 6243' 'T VISITA DISNEY CARS SANTISTA'
 'T VISITA DISNEY BELA SANTISTA']
grupo => 7
['T VISITA P BORDA DESIRE AZUL 6238']


In [48]:
sorted(df["Grupo"].unique())[-1]

7.0

In [49]:
df[["Preço", "Vendas Reais Qtde", "Grupo"]].corr()

,Preço,Vendas Reais Qtde,Grupo
Preço,1.000000,-0.069071,0.033205
Vendas Reais Qtde,-0.069071,1.000000,-0.239822
Grupo,0.033205,-0.239822,1.000000


In [50]:
df[["CMV", "CMV/QTD", "Vendas Reais Qtde"]].corr()

,CMV,CMV/QTD,Vendas Reais Qtde
CMV,1.000000,0.196381,0.664583
CMV/QTD,0.196381,1.000000,-0.056368
Vendas Reais Qtde,0.664583,-0.056368,1.000000


## Função

In [51]:
def agrupamentos(dataframe, threshold:float):
    """
    Descreve uma função que classifica os itens em grupos de similaridade utilizando fuzzy matching.
    
    dataframe -> dataframe que contém os itens a serem classificados]
    threshold -> grau de similaridade entre os nomes dos itens (0 a  100)
    """
    
    # Cria um vetor de itens únicos ordenados alfabeticamente
    lista_unicos = sorted(list(dataframe["SAP+NOME"].unique()))
    
    # Cria uma feature de NaN's chamada Grupo
    dataframe["Grupo"] = np.nan
    grupo = 0
    
    # Percorre o nosso dataframe e faz o agrupamento entre os itens
    while len(lista_unicos) > 0:
        item = lista_unicos[0] 
        similaridades = process.extract(item, lista_unicos)
        for i in range(len(similaridades)):
            if (similaridades[i][1] >= threshold):
                dataframe.loc[dataframe["SAP+NOME"] == similaridades[i][0], "Grupo"] = grupo
                lista_unicos.remove(similaridades[i][0])
        grupo += 1
    
    return dataframe

Fazendo ajustes nos grupos onde os produtos agregados não fazem sentido:
1. **Kit cabide c/3 madeira rev**

In [54]:
df_agrupado = agrupamentos(df, 55)
df_agrupado.loc[df_agrupado["SAP+NOME"] == 'KIT CABIDE C/3 MADEIRA REV', "Grupo"] = \
                                                                                (sorted(df_agrupado["Grupo"].unique())[-1]) + 1

In [56]:
df_agrupado

,SAP+NOME,UF,Semana,Mes,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,...,CMV/QTD,Região,DPPsic,Cont,Produto,Correlacao,NObs,Grupo,Semana_Ano,PDia_Semana
0,T VISITA P BORDA DESIRE AZUL 6238,AL,1,1,2017,4.74,0.482330,48.233032,4.74000,7.0,...,1.777143,Nordeste,1,1,T VISITA P BORDA DESIRE AZUL 6238,-0.140144,26.0,7.0,2017-1,2017-01-02
1,T VISITA P BORDA DESIRE AGUA 6243,AL,1,1,2017,4.99,0.535633,53.563281,4.99000,7.0,...,1.710000,Nordeste,1,1,T VISITA P BORDA DESIRE AGUA 6243,0.000750,28.0,7.0,2017-1,2017-01-02
2,AMACIANTE COMFORT CONCENTRADO 500ML,AL,1,1,2017,7.99,0.404144,40.414445,7.96619,99.0,...,4.316162,Nordeste,0,1,AMACIANTE COMFORT CONCENTRADO 500ML,-0.537418,44.0,0.0,2017-1,2017-01-02
3,T VISITA P BORDA DESIRE AZUL 6238,AM,1,1,2017,4.74,0.587795,58.779525,4.74000,4.0,...,1.590000,Norte,1,1,T VISITA P BORDA DESIRE AZUL 6238,-0.672686,38.0,7.0,2017-1,2017-01-02
4,T VISITA P BORDA DESIRE AGUA 6243,AM,1,1,2017,4.99,0.621958,62.195832,4.99000,15.0,...,1.553333,Norte,1,1,T VISITA P BORDA DESIRE AGUA 6243,-0.184839,46.0,7.0,2017-1,2017-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9415,T VISITA DISNEY CARS SANTISTA,TO,52,12,2017,4.99,0.457875,45.787546,4.99000,3.0,...,1.973333,Norte,1,1,T VISITA DISNEY CARS SANTISTA,-0.047837,15.0,6.0,2017-52,2017-12-25
9416,PANO SCOTT DURAMAX LIMPEZA DIARIA C/5,TO,52,12,2017,7.99,0.516757,51.675713,7.99000,4.0,...,2.812500,Norte,1,1,PANO SCOTT DURAMAX LIMPEZA DIARIA C/5,-0.687524,7.0,5.0,2017-52,2017-12-25
9417,FRONHA AVULSA ESTAMPADA CAMESA,TO,52,12,2017,4.99,0.516830,51.683022,4.99000,14.0,...,1.755714,Norte,1,1,NaN,NaN,NaN,3.0,2017-52,2017-12-25
9418,CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML,TO,52,12,2017,3.99,0.179620,17.962001,3.95000,14.0,...,2.360000,Norte,0,1,CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML,-0.496591,10.0,1.0,2017-52,2017-12-25


## Adicionando coluna de datas

In [57]:
### Concatenando as colunas de Ano e Semana
df_agrupado['Semana_Ano'] = df_agrupado['Ano'].map(str) + '-' + df['Semana'].map(str)

## Função lambda para percorrer a coluna 'Semana_Ano' aplicar na coluna 'PDia_Semana' o valor do primeiro dia da semana
df_agrupado['PDia_Semana'] = df_agrupado['Semana_Ano'].apply(lambda x: datetime.datetime.strptime(x + '-1', "%Y-%W-%w"))

In [58]:
df_agrupado

,SAP+NOME,UF,Semana,Mes,Ano,preço2,Mg %,Margem Líquida Real %,Preço,Vendas Reais Qtde,...,CMV/QTD,Região,DPPsic,Cont,Produto,Correlacao,NObs,Grupo,Semana_Ano,PDia_Semana
0,T VISITA P BORDA DESIRE AZUL 6238,AL,1,1,2017,4.74,0.482330,48.233032,4.74000,7.0,...,1.777143,Nordeste,1,1,T VISITA P BORDA DESIRE AZUL 6238,-0.140144,26.0,7.0,2017-1,2017-01-02
1,T VISITA P BORDA DESIRE AGUA 6243,AL,1,1,2017,4.99,0.535633,53.563281,4.99000,7.0,...,1.710000,Nordeste,1,1,T VISITA P BORDA DESIRE AGUA 6243,0.000750,28.0,7.0,2017-1,2017-01-02
2,AMACIANTE COMFORT CONCENTRADO 500ML,AL,1,1,2017,7.99,0.404144,40.414445,7.96619,99.0,...,4.316162,Nordeste,0,1,AMACIANTE COMFORT CONCENTRADO 500ML,-0.537418,44.0,0.0,2017-1,2017-01-02
3,T VISITA P BORDA DESIRE AZUL 6238,AM,1,1,2017,4.74,0.587795,58.779525,4.74000,4.0,...,1.590000,Norte,1,1,T VISITA P BORDA DESIRE AZUL 6238,-0.672686,38.0,7.0,2017-1,2017-01-02
4,T VISITA P BORDA DESIRE AGUA 6243,AM,1,1,2017,4.99,0.621958,62.195832,4.99000,15.0,...,1.553333,Norte,1,1,T VISITA P BORDA DESIRE AGUA 6243,-0.184839,46.0,7.0,2017-1,2017-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9415,T VISITA DISNEY CARS SANTISTA,TO,52,12,2017,4.99,0.457875,45.787546,4.99000,3.0,...,1.973333,Norte,1,1,T VISITA DISNEY CARS SANTISTA,-0.047837,15.0,6.0,2017-52,2017-12-25
9416,PANO SCOTT DURAMAX LIMPEZA DIARIA C/5,TO,52,12,2017,7.99,0.516757,51.675713,7.99000,4.0,...,2.812500,Norte,1,1,PANO SCOTT DURAMAX LIMPEZA DIARIA C/5,-0.687524,7.0,5.0,2017-52,2017-12-25
9417,FRONHA AVULSA ESTAMPADA CAMESA,TO,52,12,2017,4.99,0.516830,51.683022,4.99000,14.0,...,1.755714,Norte,1,1,NaN,NaN,NaN,3.0,2017-52,2017-12-25
9418,CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML,TO,52,12,2017,3.99,0.179620,17.962001,3.95000,14.0,...,2.360000,Norte,0,1,CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML,-0.496591,10.0,1.0,2017-52,2017-12-25


## Sobrescrevendo o dataset final

In [15]:
#df_agrupado.to_csv("../datasets/df_tratado_final.csv", index = False)

In [63]:
df_agrupado = df_agrupado.loc[df_agrupado.duplicated(subset = ['SAP+NOME', 'UF', 'PDia_Semana']) == False]

In [64]:
df_agrupado.to_csv("../datasets/df_tratado_final_v1.csv", index = False)